# L1 — Model, Prompts, and Output Parsing (LangChain v1 + OpenAI Responses)

# Setup

This notebook uses **OpenAI (Python SDK v2) + LangChain v1**.

## Prereqs
1. Set your API key in the environment:

```bash
export OPENAI_API_KEY="..."
```

2. Restart the kernel after setting env vars.


In [1]:
import os

# Make sure your key is set
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY in your environment before running."

MODEL = "gpt-5-mini"


## 1) Direct OpenAI call (Responses API)

In [2]:
from openai import OpenAI

client = OpenAI()

resp = client.responses.create(
    model=MODEL,
    input="What is 1+1? Answer with just the number."
)

print(resp.output_text)


2


## 2) LangChain ChatOpenAI + Prompt Templates

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model=MODEL)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{question}")
])

chain = prompt | llm

msg = chain.invoke({"question": "Explain 1+1 in one sentence."})
print(msg.content)


1 + 1 equals 2 because when you add one unit to another unit you have two units.


## 3) Structured output with Pydantic

In [7]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class MathAnswer(BaseModel):
    answer: int = Field(..., description="The numeric answer.")
    explanation: str = Field(..., description="Short explanation.")

parser = PydanticOutputParser(pydantic_object=MathAnswer)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Return JSON only. Follow the format instructions strictly.{format_instructions}"),
    ("user", "Question: {question}")
]).partial(format_instructions=parser.get_format_instructions())

chain = prompt | llm | parser

out = chain.invoke({"question": "What is 1+1?"})
print(out)
print(out.answer, out.explanation)


answer=2 explanation='1 plus 1 equals 2 by basic arithmetic (adding two ones).'
2 1 plus 1 equals 2 by basic arithmetic (adding two ones).


## 4) Tool-free 'prompt parser' pattern (robust extraction)
If the model returns extra text, parse with a schema and retry.

In [8]:
from langchain_core.runnables import RunnableLambda

def robust_parse(question: str, max_tries: int = 2):
    last_err = None
    for _ in range(max_tries):
        try:
            return chain.invoke({"question": question})
        except Exception as e:
            last_err = e
    raise last_err

print(robust_parse("What is 7+5?"))


answer=12 explanation='7 + 5 = 12.'
